# U.S. Medical Insurance Costs

## NB Purpose
This portfolio project is the culmination of the 1st part of Data Science Fundamentals within the Data Science "Analytics" track Codecademy course.

The purpose of this notebook is to act as a dev, experiment, and analysis envrionment for the data set provided for the project.
I was given a .csv file and told to analyze the information in Jupyter NB, so here we are.

## Scope:
The data set is organized with 7 descriptive fields: Age, Sex (M,F), Body Mass Index (BMI), Number of Children (Children), Smoker (text binary), region (Assuming this is representative of US regions), and Charges (Yearly Premium???)

Possible topics of investigation:
- Avg age of individuals within the data set
- Avg cost of coverage for smokers
- Relationship of Region to Smoker status
- Relationship of Region to BMI
- Relationship of Age to BMI
- Avg age of individuals with:
  - 1 child
  - 2 children
  - 3 children
  - 4 or more children
- Percentage of smokers with children vs no children


In [2]:
# Import block for required mods
import functions.insurance_functions as my_functions
import csv
import json

# Load csv data into dictionary
data_list = []
with open('./data/insurance.csv', 'r') as insurance_file_obj:
    csv_reader = csv.DictReader(insurance_file_obj, delimiter=',')
    for record in csv_reader:
        data_list.append(record)
# A dictionary would be ideal for this data set, but given each record has no key identifying this information; an indexed list will have to suffice


## Avg Age of Individuals within the Data Set
Calls the function to find the information and prints for markdown purposes

In [3]:
average_age_of_dataset = round(find_average_age_total(data_list), 2)
print(f"The average age across the entire health insurance data set is: {average_age_of_dataset}")

The average age across the entire health insurance data set is: 39.21
